In [35]:
import numpy as np
import pandas as pd
from PIL import Image

# 数据预处理

## 核心函数
#### 截取图片中心部分，并计算颜色矩

In [36]:
def img_resize(length,width,data_r):
    '''
    裁剪图片
    :param length:
    :param width:
    :param data_r:
    :return:
    '''
    m=100#保留的区域
    n=100
    l_begin=np.int64(np.round(length/2,0)-m/2)#转化为整数
    l_last=np.int64(np.round(length/2,0)+m/2)
    w_begin=np.int64(np.round(width/2,0)-n/2)
    w_last=np.int64(np.round(width/2,0)+n/2)
    data_rc=data_r[l_begin:l_last,w_begin:w_last]
    return data_rc


#一阶矩
def m1(data_rc:np.array):
    '''
    计算一阶矩
    :param data_rc:
    :return:
    '''
    return np.mean(data_rc)

#二阶矩
def m2(data_rc:np.array):
    '''
    计算二阶矩
    :param data_rc:
    :return:
    '''
    return np.std(data_rc)

#三阶矩
def var3(length,width,data_rc:np.array):
    '''
    计算三阶矩
    :param length:
    :param width:
    :param data_rc:
    :return:
    '''
    mean = np.mean(data_rc)
    data_rc = data_rc-mean
    data_rc = data_rc**3
    data_rc = np.abs(data_rc)
    data_rc = np.sum(data_rc)/(length*width)
    data_rc = data_rc**(1/3)
    return data_rc

# 计算三个通道的三个颜色矩
def get_mu(img):
    '''
    返回data 三个通道的三个颜色矩
    :param img:
    :return:
    '''
    data=np.array(img)
    length=data.shape[0]
    width=data.shape[1]
    ans = []
    for i in [0,1,2]:
        data_r=data[:,:,i]#三维所有行列
        data_rc = img_resize(length,width,data_r)
        data_rc=data_rc/255
        ans.extend([m1(data_rc),m2(data_rc),var3(length,width,data_rc)])
    return ans



In [37]:
image=r"data/images/1_1.jpg"
#打开图片
img=Image.open(image)
data=np.array(img)
data.shape



(1704, 2272, 3)

In [38]:
# #显示图片
# #im.show()
# print(data.shape)#大小，取中间的点
# data_r=data[:,:,0]#三维所有行列
# img1=img.fromarray(data_r)#第一个通道
# # img1.show()#黑白

## 获取文件夹下所有文件

In [39]:
import os
path =r"data/images"

def get_allfile(path):  # 获取所有文件
    all_file = []
    for f in os.listdir(path):  #listdir返回文件中所有目录
        f_name = os.path.join(path, f)
        all_file.append(f_name)
    return all_file

all_file=get_allfile(path)  #tickets要获取文件夹名


## 进行图片预处理，将其存入Dataframe

In [40]:
from tqdm import tqdm
#处理 文件夹下所有图片

rows_list = [] #存数据


for filename in tqdm(all_file):
    row =[]
    #打开图片
    img=Image.open(filename)

    #计算颜色矩并存入row
    row.extend(get_mu(img))

    #存入序号和类别
    row.extend([int(filename.split('.')[0].split('_')[1]),int(filename.split('.')[0].split('_')[0][-1])])
    rows_list.append(row)

# 创建dataframe 对象
columns = ['R通道一阶矩','R通道二阶矩','R通道三阶矩','G通道一阶矩','G通道二阶矩','G通道三阶矩','B通道一阶矩','B通道二阶矩','B通道三阶矩','序号','类别']
img_df = pd.DataFrame(rows_list,columns=columns)
#存储文件的路径
save_path=r'data/images_excel/imgs.csv'
img_df.to_csv(save_path,encoding='utf-8')
print('处理完成！')


100%|██████████| 203/203 [00:25<00:00,  8.10it/s]

处理完成！


# 支持向量机

In [ ]:
from sklearn import svm
X = [[0, 0], [1, 1]]
y = [0, 1]
clf = svm.SVC(gamma='scale')
clf.fit(X, y)
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
max_iter=-1, probability=False, random_state=None, shrinking=True,
tol=0.001, verbose=False)

In [1]:
!pip install scipy

ERROR: Could not install packages due to an OSError: HTTPSConnectionPool(host='files.pythonhosted.org', port=443): Max retries exceeded with url: /packages/f3/9f/80522344838ae24cac9e945240436269cbb92349f7f1f4c9dfc10cb6bad5/scipy-1.5.4-cp36-cp36m-win_amd64.whl (Caused by ConnectTimeoutError(<pip._vendor.urllib3.connection.HTTPSConnection object at 0x0000015319B6F4A8>, 'Connection to files.pythonhosted.org timed out. (connect timeout=15)'))



In [40]:
# coding=utf-8
from scipy.optimize import minimize
import numpy as np

class SVM():
	def __init__(self,data,label,eps=0.00001):
		self.data=data
		self.label=label
		self.eps=eps

	def fun(self):
		n=self.data.shape[0]
		m=self.data.shape[1]
		if(n!=len(self.label)):
			print("error :data and label should be the same dimension!")
		y=np.repeat(self.label.reshape(n,1),m,axis=1)
		A=np.multiply(self.data,y)
		A=np.dot(A,A.T)
		v=lambda alpha: 1/2*np.dot(alpha,np.dot(A,alpha))-np.sum(alpha)
		return v
	def createfun(self,i):
		return (lambda alpha:alpha[i])

	def con(self):
		C=10.0
		cons=list()
		#cons1=tuple()
		y=self.label
		for i in range(0,len(y)):
			cons.append({'type': 'ineq', 'fun': self.createfun(i)})
			#cons.append({'type': 'ineq', 'fun': lambda alpha:alpha[i]})
			#cons.append({'type': 'ineq', 'fun': lambda alpha,i=i:alpha[i]})
			#cons.append({'type': 'ineq', 'fun': lambda alpha: C-alpha[i]})
		cons.append({'type': 'eq', 'fun': lambda alpha: np.dot(alpha,y)})
		cons=tuple(cons)
		#cons=({'type': 'ineq', 'fun': lambda x: x[0]},)+({'type': 'ineq', 'fun': lambda x: x[1]},)+({'type': 'ineq', 'fun': lambda x: x[2]},)+({'type': 'eq', 'fun': lambda x: np.dot(x,y)},)
		return cons

	def fit(self,x0):
		cons=self.con()
		res=minimize(self.fun(),x0,method='SLSQP',constraints=cons)
		print(res.fun)
		print(res.success)
		print(res.x)
		eps=self.eps
		self.alpha=res.x
		ay=np.multiply(self.alpha,self.label)
		ay=ay.reshape(len(ay),1)
		ay=np.repeat(ay,self.data.shape[1],axis=1)
		ay_data=np.multiply(self.data,ay)
		self.w=np.sum(ay_data,axis=0)
		b=[]
		sv=self.alpha>eps
		self.sv=np.argwhere(sv)
		y=self.label[sv]
		x=self.data[sv,:]
		for i in  range(0,x.shape[0]):
			bi=1/y[i]-np.dot(self.w,x[i,:])
			b.append(bi)
		self.b=np.mean(b)


	def predict(self,x):
		#print('not implemented yet')
		v=np.dot(self.w,x)+self.b
		if v>=0:
			return 1
		else:
			return -1


if __name__ == "__main__":
	dataset=np.array([[3,3,1],[4,3,1],[1,1,-1],[1,2,-1]])
	data=dataset[:,:-1]
	label=dataset[:,-1]
	svm=SVM(data,label)
	x0=np.asarray((0,0,0,0))
	svm.fit(x0)
    x=[1,2]
    svm.predict(x) #




In [41]:
A = r'data/images\\1_1.jpg'
category =int(A.split('.')[0].split('_')[0][-1])
order =int(A.split('.')[0].split('_')[1])


# 图像轮廓

In [42]:
# import cv2
# image=r"data/images/1_1.jpg"
# img = cv2.imread(image)
# cv2.imshow("img", img)
#
#
# # 转换为灰度图像
# img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#
# # 转换为二值图
# ret, binary = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)
# # 获取图像的轮廓参数
# contours, hierarchy = cv2.findContours(binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
#
# for i in range(len(contours)):
#     print(str(i),cv2.moments(contours[i]))

# 图像颜色矩

In [43]:
# cv2.calcHist(images, channels, mask, histSize, ranges[, hist[, accumulate]])
# 其中images 可为单张或多张图像的array
# channels 为要计算的通道数
# mask 为图像掩膜
# histSize 为直方图的柱子数量，即将数据分布在多少个区间上计数
# range 为直方图取值范围
# hist为返回值，不用填
# accumulate 多张图的时候是否叠加
# 所以一般调用的时候只需要填上面四个参数,掩膜为None，范围0.0-255.0，数量255个：
# hist = cv2.calcHist(img, [0]， None，[256], [0.0,255.0])
#


## 颜色矩代码实现

In [44]:
# # import the necessary packages
# import numpy as np
# import cv2
#
# def color_moments(filename):
#     img = cv2.imread(filename)
#     if img is None:
#         return
#     # Convert BGR to HSV colorspace
#     hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#     # Split the channels - h,s,v
#     h, s, v = cv2.split(hsv)
#     # Initialize the color feature
#     color_feature = []
#     # N = h.shape[0] * h.shape[1]
#     # The first central moment - average
#     h_mean = np.mean(h)  # np.sum(h)/float(N)
#     s_mean = np.mean(s)  # np.sum(s)/float(N)
#     v_mean = np.mean(v)  # np.sum(v)/float(N)
#     color_feature.extend([h_mean, s_mean, v_mean])
#     # The second central moment - standard deviation
#     h_std = np.std(h)  # np.sqrt(np.mean(abs(h - h.mean())**2))
#     s_std = np.std(s)  # np.sqrt(np.mean(abs(s - s.mean())**2))
#     v_std = np.std(v)  # np.sqrt(np.mean(abs(v - v.mean())**2))
#     color_feature.extend([h_std, s_std, v_std])
#     # The third central moment - the third root of the skewness
#     h_skewness = np.mean(abs(h - h.mean())**3)
#     s_skewness = np.mean(abs(s - s.mean())**3)
#     v_skewness = np.mean(abs(v - v.mean())**3)
#     h_thirdMoment = h_skewness**(1./3)
#     s_thirdMoment = s_skewness**(1./3)
#     v_thirdMoment = v_skewness**(1./3)
#     color_feature.extend([h_thirdMoment, s_thirdMoment, v_thirdMoment])
#     return color_feature
